## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
pd.options.mode.chained_assignment = None

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.42,19.23,52.00,78,65,5.01,light rain
1,1,San Jeronimo,PE,-11.95,-75.28,68.63,24,11,3.80,few clouds
2,2,Mirnyy,RU,62.54,113.96,50.00,53,100,6.71,overcast clouds
3,3,Cape Town,ZA,-33.93,18.42,54.00,87,20,5.82,few clouds
4,4,Tuktoyaktuk,CA,69.45,-133.04,48.20,66,20,12.75,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your vacation? '))
max_temp = float(input('What is the maximum temperature you would like for your vacation '))

What is the minimum temperature you would like for your vacation? 80
What is the maximum temperature you would like for your vacation 95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                      (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
18,18,Road Town,VG,18.42,-64.62,87.80,70,40,9.17,scattered clouds
19,19,North Platte,US,41.12,-100.77,87.80,35,1,20.80,clear sky
22,22,Kavaratti,IN,10.57,72.64,82.09,77,100,6.89,light rain
24,24,Talavera De La Reina,ES,39.96,-4.83,87.01,17,0,14.67,clear sky
26,26,Avera,US,33.19,-82.53,89.60,66,75,5.82,broken clouds
33,33,Ouallam,NE,14.32,2.09,88.56,52,12,10.09,few clouds
40,40,Qandala,SO,11.47,49.87,89.98,51,100,1.39,overcast clouds
44,44,Georgetown,MY,5.41,100.34,80.60,88,20,1.57,few clouds
45,45,Westport,US,41.14,-73.36,87.01,57,20,8.05,few clouds
60,60,La Palma,US,33.85,-118.05,86.00,42,1,4.70,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                165
City                   165
Country                165
Lat                    165
Lng                    165
Max Temp               165
Humidity               165
Cloudiness             165
Wind Speed             165
Current Description    165
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
cities_nan_value = float("NaN")
preferred_cities_df.replace("",cities_nan_value, inplace=True)
preferred_cities_clean_df = preferred_cities_df.dropna(how='any', inplace=False)
preferred_cities_clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
18,18,Road Town,VG,18.42,-64.62,87.80,70,40,9.17,scattered clouds
19,19,North Platte,US,41.12,-100.77,87.80,35,1,20.80,clear sky
22,22,Kavaratti,IN,10.57,72.64,82.09,77,100,6.89,light rain
24,24,Talavera De La Reina,ES,39.96,-4.83,87.01,17,0,14.67,clear sky
26,26,Avera,US,33.19,-82.53,89.60,66,75,5.82,broken clouds
33,33,Ouallam,NE,14.32,2.09,88.56,52,12,10.09,few clouds
40,40,Qandala,SO,11.47,49.87,89.98,51,100,1.39,overcast clouds
44,44,Georgetown,MY,5.41,100.34,80.60,88,20,1.57,few clouds
45,45,Westport,US,41.14,-73.36,87.01,57,20,8.05,few clouds
60,60,La Palma,US,33.85,-118.05,86.00,42,1,4.70,clear sky


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Road Town,VG,87.80,scattered clouds,18.42,-64.62,
19,North Platte,US,87.80,clear sky,41.12,-100.77,
22,Kavaratti,IN,82.09,light rain,10.57,72.64,
24,Talavera De La Reina,ES,87.01,clear sky,39.96,-4.83,
26,Avera,US,89.60,broken clouds,33.19,-82.53,
33,Ouallam,NE,88.56,few clouds,14.32,2.09,
40,Qandala,SO,89.98,overcast clouds,11.47,49.87,
44,Georgetown,MY,80.60,few clouds,5.41,100.34,
45,Westport,US,87.01,few clouds,41.14,-73.36,
60,La Palma,US,86.00,clear sky,33.85,-118.05,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():   
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except(IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Road Town,VG,87.80,scattered clouds,18.42,-64.62,"ZINGARA 76ft Catamaran, full crew-all included"
19,North Platte,US,87.80,clear sky,41.12,-100.77,Holiday Inn Express & Suites North Platte
22,Kavaratti,IN,82.09,light rain,10.57,72.64,Bismillah Hotel And Lodging
24,Talavera De La Reina,ES,87.01,clear sky,39.96,-4.83,Be Live City Center Talavera
26,Avera,US,89.60,broken clouds,33.19,-82.53,
33,Ouallam,NE,88.56,few clouds,14.32,2.09,
40,Qandala,SO,89.98,overcast clouds,11.47,49.87,
44,Georgetown,MY,80.60,few clouds,5.41,100.34,Cititel Penang
45,Westport,US,87.01,few clouds,41.14,-73.36,The Westport Inn
60,La Palma,US,86.00,clear sky,33.85,-118.05,La Quinta Inn & Suites by Wyndham Buena Park


In [11]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("",nan_value, inplace=True)
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'], inplace=False)

In [12]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Road Town,VG,87.80,scattered clouds,18.42,-64.62,"ZINGARA 76ft Catamaran, full crew-all included"
19,North Platte,US,87.80,clear sky,41.12,-100.77,Holiday Inn Express & Suites North Platte
22,Kavaratti,IN,82.09,light rain,10.57,72.64,Bismillah Hotel And Lodging
24,Talavera De La Reina,ES,87.01,clear sky,39.96,-4.83,Be Live City Center Talavera
44,Georgetown,MY,80.60,few clouds,5.41,100.34,Cititel Penang
45,Westport,US,87.01,few clouds,41.14,-73.36,The Westport Inn
60,La Palma,US,86.00,clear sky,33.85,-118.05,La Quinta Inn & Suites by Wyndham Buena Park
65,Freeport,US,89.01,scattered clouds,40.66,-73.58,The Freeport Inn and Marina
68,Homestead,US,93.20,broken clouds,25.47,-80.48,Best Western Gateway to the Keys
69,Djibo,BF,86.49,clear sky,14.10,-1.63,Centre Des Ressources DPENA


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

In [21]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# 11a. Add a marker layer for each city to the map. 
# Get the maximum temperature.
max_temp = clean_hotel_df['Max Temp']

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
                                dissipating=False, max_intensity=300,
                                point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the heatmap layer.
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))